<a href="https://colab.research.google.com/github/nadhifahlutfiyah/Tugas7_BigData/blob/main/slide30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=b6c51236a58121c1ec46783b2f02499a8554596ab23e7bc757a9952a308f25b9
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.recommendation import ALS

In [9]:
spark = SparkSession.builder.appName("Movie Lens").getOrCreate()

In [18]:
class Rating:
    def __init__(self, userId, movieId, rating, timestamp):
        self.userId = int(userId)
        self.movieId = int(movieId)
        self.rating = float(rating)
        self.timestamp = float(timestamp)

def parseRating(str):
    fields = str.split(",")
    assert (len(fields) == 4)
    return Rating(int(fields[0]), int(fields[1]), float(fields[2]), int(fields[3]))

In [19]:
parseRating("1,1193,5,978300760")

In [20]:
ratings = spark.read.csv("/content/drive/MyDrive/Big Data/ratings.csv", header=True, inferSchema=True)
ratings.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [21]:
training, test = ratings.randomSplit([0.8, 0.2])

als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")

model = als.fit(training)
model.write().overwrite().save("mymodel")

In [22]:
predictions = model.transform(test)
squaredErrors = predictions.withColumn("squaredError", (col("rating") - col("prediction")) ** 2)
mse = squaredErrors.filter(~col("squaredError").isNull()).selectExpr("sum(squaredError) as sumSquaredError").collect()[0][0]

In [23]:
predictions.show(10)

+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|     1|    362|   5.0|964982588|  4.121773|
|     1|    333|   5.0|964981179|  3.757859|
|     1|     47|   5.0|964983815|  4.999283|
|     1|    235|   4.0|964980908| 4.3132267|
|     1|    163|   5.0|964983650|  4.333357|
|     1|    553|   5.0|964984153|  3.683965|
|     1|    316|   3.0|964982310|  3.769468|
|     1|    423|   3.0|964982363|  2.520809|
|     1|    151|   5.0|964984041| 4.7304077|
|     1|    592|   4.0|964982271| 3.9564989|
+------+-------+------+---------+----------+
only showing top 10 rows

